# 要約 
このノートブックでは、`dictionaryapi`を利用してコンペティションに新しく追加されたキーワードの定義を取得し、これらの定義を基に以下の目的を達成することを目指しています：

1. **トレーニング用の良質な質問の生成**: 既知の回答を持つ質問を作成し、トレーニングに活用します。
2. **キーワード“things”の理解促進**: このキーワードが何であるかをより深く理解し、関連するアイデアを増やします。
3. **回答者エージェントへの情報提供**: `answerer`という役割のエージェントに対して、より多くの情報を与え、正確な応答を引き出すようにします。

ノートブック内では、以下のリソースと手法が使用されています：

- **ライブラリ**: `pandas`, `requests`, `json`, `jq`, `random`を使用し、データの取得と処理を行っています。
- **APIアクセス**: `dictionaryapi`のREST APIを介してキーワードの定義を取得しています。
- **データ処理**: 取得したJSONデータを`jq`を使ってフィルタリングし、必要な情報を抽出します。また、エラーハンドリング機能を持つ定義生成器を定義し、時間制限を設けてリクエストを送ることで、タイムアウトを回避しています。

最終結果は、物のキーワードに対する定義を含む`things_definitions.json`ファイルに保存されます。この結果を用いて、質問生成やエージェントの情報提供において、より効果的な戦略が実現できることが期待されています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語の簡単な解説を示します。

1. **dictionaryapi**:
   - オンライン辞書APIの一つで、特定の単語に関する定義や情報を取得するために使用される。競技プログラムやダイナミックなデータベースへのアクセスを可能にする。

2. **RAG (Retrieval-Augmented Generation)**:
   - 検索と生成のハイブリッドアプローチで、特定の情報を文脈に応じて検索し、その情報を使用してコンテンツを生成する手法。特に、モデルが豊富な文脈情報を持つ場合に有用。

3. **jq**:
   - JSONデータを処理するための軽量なコマンドラインツールで、データのフィルタリングや変形に使用される。このノートブックでは、APIからの応答を簡単に扱うために使用されている。

4. **curl**:
   - URLを使用してデータを取得するためのコマンドラインツールで、APIへのHTTPリクエストを簡単に行うことができる。データの取得や送信に広く利用されている。

5. **meanings**:
   - APIから取得したJSONデータで、特定の単語に対する定義のリストを含むフィールド。通常、同じ単語に対する複数の意味を持つことがある。

6. **JSON (JavaScript Object Notation)**:
   - データ交換形式の一つで、構造化されたデータを軽量なテキスト形式で表現する方法。APIからのレスポンスデータなどで広く使用されている。

7. **status_code**:
   - HTTPレスポンスのコードを示すもので、リクエストが正常に処理されたかどうかを判断するために使用される。例: `200`は成功、`404`は見つからない、`500`はサーバーエラーを示す。

8. **エージェント**:
   - 特定の仕事やタスクを実行するためのプログラムやロジックを指すことがある。ここでは、質問者や回答者の役割を担うモデルがエージェントとして扱われている。

9. **フィルタリング**:
   - データの中から特定の条件に基づいて必要な情報を選び出すプロセス。APIレスポンスの中から意味のある部分だけを取り出すのに使われる。

10. **タイムアウト**:
    - プログラムが応答を待つ際に設定された制限時間を超えた場合に、そのプロセスを強制的に終了すること。APIリクエストなどで、応答が遅れる場合に重要な考慮事項。

これらの用語は、機械学習や深層学習の実務においても重要な概念であり、理解を深めることでノートブックや関連するコードをより容易に扱うことができるでしょう。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

In this notebook we use `dictionaryapi` to find definitions for the newly added keywords in the [competition](https://www.kaggle.com/competitions/llm-20-questions)  
The goal is to use these definitions to:
1. Generate (good) questions with known answers for training purpuse
1. Understand better what can be keyword `things`, so we can generate more
1. Give more information to the agent with role `answerer` so it can respond more accurately (RAG ?)

results are store in `things_definitions.json`  
Note: 
- Some word have multiple definitions (Ex:`Plate` has many)
- For most of two or more parts words (Ex:`Air compressor`) dictionaryapi has no definition. Some two parts keyword have definitions (Ex:`Contact lenses`)
- Almost all one-word keywords have atleast one definition (exceptions: RV, IV,...)


</div>
<div class="column-right">

# 日本語訳

このノートブックでは、`dictionaryapi`を使用して、[コンペティション](https://www.kaggle.com/competitions/llm-20-questions)に新しく追加されたキーワードの定義を見つけます。  
目標は、これらの定義を利用して以下のことを行うことです：
1. トレーニング目的のために、既知の回答を持つ（良い）質問を生成すること
2. キーワード`things`が何であるかをよりよく理解し、より多く生成できるようにすること
3. `answerer`という役割のエージェントにより多くの情報を提供し、より正確に応答できるようにすること（RAG ?）

結果は`things_definitions.json`に保存します。  
注意：
- 一部の単語には複数の定義があります（例: `Plate`には多くの意味があります）
- ほとんどの二つ以上の部分からなる単語（例: `Air compressor`）にはdictionaryapiには定義がありません。いくつかの二部分のキーワードには定義があります（例: `Contact lenses`）。
- ほとんどの一単語のキーワードには少なくとも一つの定義があります（例外: RV, IV,...）



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%capture
!pip install jq
```

</div>
<div class="column-right">

# 日本語訳

```python
%%capture
!pip install jq
```

</div>
</details>

In [ ]:
%%capture
!pip install jq

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd
import requests
import json
import time
import jq
import random
```

</div>
<div class="column-right">

# 日本語訳

```python
import pandas as pd
import requests
import json
import time
import jq
import random
```

</div>
</details>

In [ ]:
import pandas as pd
import requests
import json
import time
import jq
import random

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
keywords = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/keywords.csv", index_col=0)
keywords
```

</div>
<div class="column-right">

# 日本語訳

```python
keywords = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/keywords.csv", index_col=0)
keywords
```

</div>
</details>

In [ ]:
keywords = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/keywords.csv", index_col=0)
keywords

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%run /kaggle/input/llm-20-questions/llm_20_questions/keywords.py
keywords_old:set[str] = {details["keyword"] for category in json.loads(KEYWORDS_JSON) for details in category["words"]}
# [w for w in keywords.loc[keywords.category=="place","keyword"] if w not in keywords_old]
keywords_place = set(keywords.loc[keywords.category=="place","keyword"])
print(f"Current place keyword not in the previous json: {keywords_place-keywords_old}")
print(f"Old place keyword not in current: {keywords_old-keywords_place}")
del keywords_place, keywords_old
```

</div>
<div class="column-right">

# 日本語訳

```python
%run /kaggle/input/llm-20-questions/llm_20_questions/keywords.py
keywords_old:set[str] = {details["keyword"] for category in json.loads(KEYWORDS_JSON) for details in category["words"]}
# 現在のキーワードのうち、「place」カテゴリーに属するキーワードで、以前のjsonに含まれていないものをリスト表示します
# [w for w in keywords.loc[keywords.category=="place","keyword"] if w not in keywords_old]
keywords_place = set(keywords.loc[keywords.category=="place","keyword"])
print(f"現在の場所キーワード（以前のjsonにない）: {keywords_place-keywords_old}")
print(f"以前の場所キーワード（現在にない）: {keywords_old-keywords_place}")
del keywords_place, keywords_old
```

</div>
</details>

In [ ]:
%run /kaggle/input/llm-20-questions/llm_20_questions/keywords.py
keywords_old:set[str] = {details["keyword"] for category in json.loads(KEYWORDS_JSON) for details in category["words"]}
# 現在のキーワードのうち、「place」カテゴリーに属するキーワードで、以前のjsonに含まれていないものをリスト表示します
# [w for w in keywords.loc[keywords.category=="place","keyword"] if w not in keywords_old]
keywords_place = set(keywords.loc[keywords.category=="place","keyword"])
print(f"現在の場所キーワード（以前のjsonにない）: {keywords_place-keywords_old}")
print(f"以前の場所キーワード（現在にない）: {keywords_old-keywords_place}")
del keywords_place, keywords_old

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Things

</div>
<div class="column-right">

# 日本語訳

# 物の定義


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
keywords[keywords.category=="things"]
```

</div>
<div class="column-right">

# 日本語訳

```python
keywords[keywords.category=="things"]
```

</div>
</details>

In [ ]:
keywords[keywords.category=="things"]

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
url_api = "https://api.dictionaryapi.dev/api/v2/entries/en/{word}"
# test api
r = requests.get(url_api.format(word="Zinc"))
r.status_code
```

</div>
<div class="column-right">

# 日本語訳

```python
url_api = "https://api.dictionaryapi.dev/api/v2/entries/en/{word}"
# APIテスト
r = requests.get(url_api.format(word="Zinc"))
r.status_code
```

</div>
</details>

In [ ]:
url_api = "https://api.dictionaryapi.dev/api/v2/entries/en/{word}"
# APIテスト
r = requests.get(url_api.format(word="Zinc"))
r.status_code

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
r.json()
```

</div>
<div class="column-right">

# 日本語訳

```python
r.json()
```

</div>
</details>

In [ ]:
r.json()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Example jq to filter json
jq.compile('select(length > 0) []["meanings"]').input_value(r.json()).first()
```

</div>
<div class="column-right">

# 日本語訳

```python
# JSONをフィルタリングするためのjqの例
jq.compile('select(length > 0) []["meanings"]').input_value(r.json()).first()
```

</div>
</details>

In [ ]:
# JSONをフィルタリングするためのjqの例
jq.compile('select(length > 0) []["meanings"]').input_value(r.json()).first()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Alternative in bash
jq is command-line JSON processor

</div>
<div class="column-right">

# 日本語訳

## bashでの代替
jqはコマンドラインのJSONプロセッサです


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!curl "https://api.dictionaryapi.dev/api/v2/entries/en/{Wristband}" | jq '.[]["meanings"]'
```

</div>
<div class="column-right">

# 日本語訳

```python
!curl "https://api.dictionaryapi.dev/api/v2/entries/en/{Wristband}" | jq '.[]["meanings"]'
```

</div>
</details>

In [ ]:
!curl "https://api.dictionaryapi.dev/api/v2/entries/en/{Wristband}" | jq '.[]["meanings"]'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!curl "https://api.dictionaryapi.dev/api/v2/entries/en/{Plate}" | jq '.[]'
```

</div>
<div class="column-right">

# 日本語訳

```python
!curl "https://api.dictionaryapi.dev/api/v2/entries/en/{Plate}" | jq '.[]'
```

</div>
</details>

In [ ]:
!curl "https://api.dictionaryapi.dev/api/v2/entries/en/{Plate}" | jq '.[]'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
keywords.loc[keywords.category=="things","keyword"].str.len().max()
```

</div>
<div class="column-right">

# 日本語訳

```python
keywords.loc[keywords.category=="things","keyword"].str.len().max()
```

</div>
</details>

In [ ]:
keywords.loc[keywords.category=="things","keyword"].str.len().max()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Definitions from previous version
with open("/kaggle/input/20questions-keywords-things-definitions/things_definitions.json","r") as f:
    keywords_definitions = json.load(f)

print(len(keywords_definitions), "things keywords total")
keywords_definitions = {k:v for k,v in keywords_definitions.items() if len(v)>0}
print(len(keywords_definitions) , "with found definitions")
next(iter(keywords_definitions.items()))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 前のバージョンからの定義
with open("/kaggle/input/20questions-keywords-things-definitions/things_definitions.json","r") as f:
    keywords_definitions = json.load(f)

print(len(keywords_definitions), "の物のキーワードが総数")
keywords_definitions = {k:v for k,v in keywords_definitions.items() if len(v)>0}
print(len(keywords_definitions) , "が見つかった定義を持つ")

next(iter(keywords_definitions.items()))
```

</div>
</details>

In [ ]:
# 前のバージョンからの定義
with open("/kaggle/input/20questions-keywords-things-definitions/things_definitions.json","r") as f:
    keywords_definitions = json.load(f)

print(len(keywords_definitions), "の物のキーワードが総数")
keywords_definitions = {k:v for k,v in keywords_definitions.items() if len(v)>0}
print(len(keywords_definitions) , "が見つかった定義を持つ")

next(iter(keywords_definitions.items()))

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%time
def definition_generator():
    fail=0
    for num, keyword in enumerate(keywords.loc[keywords.category=="things","keyword"]):
        r = requests.get(url_api.format(word=keyword))
        if keyword in keywords_definitions:
            yield keyword, keywords_definitions[keyword]
            continue
        if r.status_code != 200:
            print(f"Error {r.status_code=} {keyword = :30} failrate:{fail/num:.2%}")
            yield keyword, []
            fail+=1
            continue
        try:
            yield keyword, jq.compile('select(length > 0) []["meanings"]').input_value(r.json()).first()
        except (ValueError, JSONDecodeError) as e:
            print(f"{type(e).__name__} {e} parsing response {keyword = } failrate:{fail/num:.2%})")
            print(e)
            fail+=1
            yield keyword, []
        time.sleep(1+random.random()/2) #slow request to not be timeout 
    print(f"{fail=} ({fail/num:.2%})")

things_definitions={keyword: definitions for keyword, definitions in definition_generator()}
len(things_definitions)
```

</div>
<div class="column-right">

# 日本語訳

```python
%%time
def definition_generator():
    fail=0
    for num, keyword in enumerate(keywords.loc[keywords.category=="things","keyword"]):
        r = requests.get(url_api.format(word=keyword))
        if keyword in keywords_definitions:
            yield keyword, keywords_definitions[keyword]
            continue
        if r.status_code != 200:
            print(f"エラー {r.status_code=} {keyword = :30} 失敗率:{fail/num:.2%}")
            yield keyword, []
            fail+=1
            continue
        try:
            yield keyword, jq.compile('select(length > 0) []["meanings"]').input_value(r.json()).first()
        except (ValueError, JSONDecodeError) as e:
            print(f"{type(e).__name__} {e} レスポンスの解析中 {keyword = } 失敗率:{fail/num:.2%})")
            print(e)
            fail+=1
            yield keyword, []
        time.sleep(1+random.random()/2) # タイムアウトを避けるためにリクエストの速度をゆっくりにする
    print(f"{fail=} ({fail/num:.2%})")

things_definitions={keyword: definitions for keyword, definitions in definition_generator()}
len(things_definitions)
```

</div>
</details>

In [ ]:
%%time
def definition_generator():
    fail=0
    for num, keyword in enumerate(keywords.loc[keywords.category=="things","keyword"]):
        r = requests.get(url_api.format(word=keyword))
        if keyword in keywords_definitions:
            yield keyword, keywords_definitions[keyword]
            continue
        if r.status_code != 200:
            print(f"エラー {r.status_code=} {keyword = :30} 失敗率:{fail/num:.2%}")
            yield keyword, []
            fail+=1
            continue
        try:
            yield keyword, jq.compile('select(length > 0) []["meanings"]').input_value(r.json()).first()
        except (ValueError, JSONDecodeError) as e:
            print(f"{type(e).__name__} {e} レスポンスの解析中 {keyword = } 失敗率:{fail/num:.2%})")
            print(e)
            fail+=1
            yield keyword, []
        time.sleep(1+random.random()/2) # タイムアウトを避けるためにリクエストの速度をゆっくりにする
    print(f"{fail=} ({fail/num:.2%})")

things_definitions={keyword: definitions for keyword, definitions in definition_generator()}
len(things_definitions)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Result
& Store

</div>
<div class="column-right">

# 日本語訳

# 結果
& 保存


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
with open("things_definitions.json","w") as f:
    json.dump(things_definitions,f)
things_definitions
```

</div>
<div class="column-right">

# 日本語訳

```python
with open("things_definitions.json","w") as f:
    json.dump(things_definitions,f)
things_definitions
```

</div>
</details>

In [ ]:
with open("things_definitions.json","w") as f:
    json.dump(things_definitions,f)
things_definitions